# 1. Setup

1.1 Install Dependencies
1.2 Import Dependencies
1.3 Set GPU Growth
1.4 Create Folder Structures

In [1]:
!pip install tensorflow opencv-python matplotlib

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.99.1 requires pydantic!=1.8,!=1.8.1,<2.0.0,>=1.7.4, but you have pydantic 1.7.3 which is incompatible.
gradio 3.42.0 requires pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,<3.0.0,>=1.7.4, but you have pydantic 1.7.3 which is incompatible.
langchain 0.0.148 requires aiohttp<4.0.0,>=3.8.3, but you have aiohttp 3.8.1 which is incompatible.
langchain 0.0.148 requires SQLAlchemy<2,>=1, but you have sqlalchemy 2.0.23 which is incompatible.
prisma 0.10.0 requires pydantic<3,>=1.8.0, but you have pydantic 1.7.3 which is incompatible.
pydantic-core 2.6.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.



   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.8 MB 9.4 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/14.8 MB 8.3 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/14.8 MB 7.0 MB/s eta 0:00:02
   --- ------------------------------------ 1.2/14.8 MB 6.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.2/14.8 MB 6.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.2/14.8 MB 6.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/14.8 MB 5.3 MB/s eta 0:00:03
   ----- ---------------------------------- 2.0/14.8 MB 4.8 MB/s eta 0:00:03
   ----- ---------------------------------- 2.2/14.8 MB 4.8 MB/s eta 0:00:03
   ------ --------------------------------- 2.3/14.8 MB 4.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.3/14.8 MB 4.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.8 MB 4.0 MB/s eta 0:00:04
   --

In [2]:
#import Standard Dependencies
import cv2
import os
import random 
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#import tensorflow dependencies - Functional API
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [4]:
#1.4 create folder structures
#setup path
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [5]:
#create the Directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH) 

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

# 2. Collect Positives and Anchors
2.1 Untar Labelled Faces in the Wild Dataset
2.2 Collect Positive and Anchor Classes

In [6]:
# http://vis.cs.umass.edu/lfw/

In [ ]:
#uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw-funneled.tgz

In [7]:
#Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw_funneled'):
    for file in os.listdir(os.path.join('lfw_funneled', directory)):
        EX_PATH = os.path.join('lfw_funneled', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [8]:
#2.2 Collect Positive and Anchor Classes
cap = cv2.VideoCapture(3)
while cap.isOpened():
    ret, frame = cap.read()
    
    cv2.imshow('Image Collection', frame)
    
    if cv2.waitkey(1) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# 3.Load and Preprocess Images